In [1]:
from ckonlpy.tag import Twitter
import pickle
import re
import sys
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from pprint import pprint
from collections import Counter
import numpy as np
import data_handler as handler
import data_analyzer as analyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import mglearn
from pprint import pprint
from ckonlpy.tag import Twitter
import numpy as np
import gc

class Social_analysis():
    
    non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

    def __init__(self):
        self.twitter = Twitter()
        
    def pickle_to_table(self, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        data = data[1:]
        for idx, i in enumerate(data):
            data[idx][2] = i[2].replace('#', ' ').translate(self.non_bmp_map)
            data[idx][3] = '/'.join(i[3])
            data[idx][4] = '/'.join(i[4])
        self.row_data = np.array(data)
  
    def hashtags_split(self, hashtags):        
        hashtags_split = []
        for i in hashtags:
            hashtags_split.append(i.split('/'))
        
        hashtags_list = []
        
        for i in hashtags_split:
            temp = []
            for j in i:
                if self.isHangul(j):
                    t_hashtags = j.translate(self.non_bmp_map)
                    temp.append(t_hashtags)
            hashtags_list.append(temp)
        self.hashtags_list = hashtags_list
        
        return hashtags_list
            

        
    def add_keyword_dic(self, keyword_list, tag='Noun'):
        for i in keyword_list:
            if len(i) == 1:
                self.twitter.add_dictionary(i[0], tag)
            else:
                self.twitter.add_dictionary(i[0], i[1])
        
    def morph_pos(self, text_list, exception_list = ['맛', '밥', '물', '몸']):
        
        morph_list = []
        noun_list = []
        adj_list = []
        verb_list = []
        
        for j in text_list:
            parsed = self.twitter.pos(j)
            temp = []
            n_temp = []
            adj_temp = []
            verb_temp = []
            
            for i in parsed:
                if self.isHangul(i[0]):
                    if len(i[0]) > 1:
                        temp.append(i)
                        if i[1] == 'Noun':
                            n_temp.append(i[0])
                        elif i[1] == 'Verb':
                            adj_temp.append(i[0])
                        elif i[1] == 'Adjective':
                            verb_temp.append(i[0])
                        elif i in exception_list:
                            temp.append(i)
                else: print(i, '한글이 아닙니다.')
            

            morph_list.append(temp)
            noun_list.append(n_temp)
            adj_list.append(adj_temp)
            verb_list.append(verb_temp)

        nav_list = noun_list + adj_list + verb_list

        return morph_list, nav_list, noun_list, adj_list, verb_list


    def word_substitute(self, dataset, sublist):
        dataset = dataset
        sub_book = dict()
        for i in sublist:
            for j in i['sub_words']:
                sub_book[j] = i['main']
        gc.collect()
        for n, i in enumerate(dataset):
            dataset[n] = [sub_book.get(item,item) for item in i]

        del sub_book
        gc.collect()

        return dataset
    
    def word_delete(dataset, del_list):
        dataset = dataset

        for n, line in enumerate(dataset):
             dataset[n] = [i for i in line if i not in del_list]

        return dataset

    
    def isHangul(self, text):
        encText = text
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
        return hanCount > 0

    

class SB_Word2Vec():    
    
    def __init__(self, morph_list):
        self.dct = Dictionary(morph_list)
        self.corpus = [self.dct.doc2bow(line) for line in morph_list]
        self.build_Word2Vec(morph_list)
    
    def make_Word2Vec(self, morph_list, size=50, window=2, min_count=10, iteration=100):
        self.em = Word2Vec(morph_list, size=size, window=window, min_count=min_count, iter=iteration)
        self.em_vocab = list(self.em.wv.vocab.keys())
        self.em_vocab_dic = {word:idx for idx, word in enumerate(self.em_vocab)}

    def make_Word2Sen_matrix(self): 
        vocab_size = len(self.em_vocab)
        self.sen_matrix = np.zeros((len(self.corpus), vocab_size))
        for idx, row in enumerate(self.sen_matrix):
            for idx2, frequency in self.corpus[idx]:
                    if self.dct[idx2] in self.em_vocab:
                        self.sen_matrix[idx][self.em_vocab_dic[self.dct[idx2]]] = frequency                
        self.sim_matrix = np.zeros((vocab_size, vocab_size))
        for idx, w1 in enumerate(self.em_vocab):
            for idx2, w2 in enumerate(self.em_vocab):
                self.sim_matrix[idx][idx2] =  self.em.wv.similarity(w1, w2)

        self.word2sen_matrix = np.dot(self.sim_matrix, np.transpose(self.sen_matrix))

        return self.word2sen_matrix

    def get_sim_sen(self, keyword, main_text_list, number=1):
        self.sim_sen_index = np.argsort(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        self.most_sim_sen_index = np.argmax(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        index_list = self.sim_sen_index.reshape((-1,)).tolist()
        index_list.reverse()
        
        for idx, i in enumerate(index_list[:number]):
            print(str(idx + 1) + '. ', main_text_list[i])
            print(main_hash_dic[main_text_list[i]])
        return index_list
    
    def build_Word2Vec(self, morph_list):
        self.make_Word2Vec(morph_list)
        self.make_Word2Sen_matrix()
        
        
class SB_LDA():

    def make_lda(self, morph_joined, ntopic=10, learning_method='batch', max_iter=25, random_state=0, n_words=20):        
        self.vect = CountVectorizer(max_features=10000, max_df=.15)
        self.X = self.vect.fit_transform(morph_joined)
        self.lda = LatentDirichletAllocation(n_components=ntopic, learning_method=learning_method, max_iter=max_iter, random_state=random_state)
        self.document_topics = self.lda.fit_transform(self.X)
        self.sorting = np.argsort(self.lda.components_, axis=1)[:, ::-1]
        self.feature_names = np.array(vect.get_feature_names())
        mglearn.tools.print_topics(topics=range(ntopic), feature_names=self.feature_names, sorting=self.sorting, topics_per_chunk=5, n_words=n_words)

    def related_doc(self, main_text_list, number):
        category = np.argsort(self.document_topics[:, number])[::-1]
        related_docs = []
        for i in category[:10]:
            print(i)
            print(main_text_list[i] + ".\n")
            related_docs.append((i, main_text_list[i]))
        return related_docs

class SB_Tfidf():    
    
    def __init__(self, list_morph_merged):
        self.list_morph_merged = list_morph_merged
        self.dct = Dictionary(self.list_morph_merged)
        self.corpus = [self.dct.doc2bow(line) for line in self.list_morph_merged]

    def get_tfidf(self):       
        self.model = TfidfModel(self.corpus)
        self.tfidf = []
        for i in self.corpus:
             self.tfidf.append(sorted(self.model[i], key = lambda x: x[1], reverse=True))
        self.tfidf_hangul = []
        for idx1, i in enumerate(self.tfidf):
            self.tfidf_hangul.append([(self.dct[j[0]], j[1]) for j in i])        
        
        return self.tfidf_hangul
    
def frequency(morph_merged):
    word_count = Counter(merged)
    word_count2 = []
    for i in word_count:
        word_count2.append((i, word_count[i]))
    word_count2 = sorted(word_count2, key=lambda x: x[1], reverse = True)
    return word_count2

C:\Users\abys\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
intake = Social_analysis()

In [3]:
intake.pickle_to_table('intake_list.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/intake_list.txt'

In [6]:
morph_list, nav_list, noun_list, adj_list, verb_list= intake.morph_pos(intake.row_data[:, 2])

AttributeError: 'Social_analysis' object has no attribute 'row_data'

In [12]:
morph_list[1] '바쁜/Adjective,아침/Noun'

[('바쁜', 'Adjective'),
 ('아침', 'Noun'),
 ('식사', 'Noun'),
 ('대용', 'Noun'),
 ('으로', 'Josa'),
 ('마시', 'Verb'),
 ('있는', 'Adjective'),
 ('가벼운', 'Adjective'),
 ('테이크', 'Noun'),
 ('밀스', 'Noun'),
 ('라이트', 'Noun'),
 ('간편하게', 'Adjective'),
 ('있고', 'Adjective'),
 ('영양소', 'Noun'),
 ('까지', 'Noun'),
 ('챙길', 'Verb'),
 ('있어', 'Adjective'),
 ('밀스', 'Noun'),
 ('라이트', 'Noun'),
 ('덕분', 'Noun'),
 ('아침', 'Noun'),
 ('거르', 'Noun'),
 ('없어', 'Adjective'),
 ('아침', 'Noun'),
 ('마다', 'Noun'),
 ('포만', 'Noun'),
 ('호로록', 'Adverb'),
 ('운동하는', 'Verb'),
 ('직장인', 'Noun'),
 ('식단', 'Noun'),
 ('인증', 'Noun'),
 ('하루', 'Noun'),
 ('시작', 'Noun'),
 ('간식', 'Noun'),
 ('타임', 'Noun'),
 ('아침', 'Noun'),
 ('스타', 'Noun'),
 ('그램', 'Noun'),
 ('간식', 'Noun'),
 ('식사', 'Noun'),
 ('대용', 'Noun'),
 ('건강', 'Noun'),
 ('스타', 'Noun'),
 ('그램', 'Noun'),
 ('건강', 'Noun'),
 ('그램', 'Noun'),
 ('식사', 'Noun'),
 ('아침식사', 'Noun'),
 ('아침', 'Noun'),
 ('스타', 'Noun'),
 ('그램', 'Noun'),
 ('맛있다', 'Adjective'),
 ('그램', 'Noun'),
 ('맛스타', 'Noun'),
 ('그램', 'Noun')]

[['아침식사',
  '모닝',
  '단호박',
  '테이크',
  '모닝',
  '아침',
  '기도',
  '입맛',
  '요거',
  '하나',
  '아침',
  '뚝딱',
  '여러',
  '종류',
  '굳이',
  '단호박',
  '고구마'],
 ['아침',
  '식사',
  '대용',
  '테이크',
  '밀스',
  '라이트',
  '영양소',
  '까지',
  '밀스',
  '라이트',
  '덕분',
  '아침',
  '거르',
  '아침',
  '마다',
  '포만',
  '직장인',
  '식단',
  '인증',
  '하루',
  '시작',
  '간식',
  '타임',
  '아침',
  '스타',
  '그램',
  '간식',
  '식사',
  '대용',
  '건강',
  '스타',
  '그램',
  '건강',
  '그램',
  '식사',
  '아침식사',
  '아침',
  '스타',
  '그램',
  '그램',
  '맛스타',
  '그램']]

In [9]:
intake.datset = handler.word_substitution(intake.dataset, [{'main': '모닝죽', 'sub_words':['단호박', '꿀고구마']}])

['아침식사', '모닝죽', '단호박', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
아침식사
['아침식사', '모닝죽', '단호박', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
모닝죽
['아침식사', '모닝죽', '단호박', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
단호박
['아침식사', '모닝죽', '모닝죽', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
인테이크
['아침식사', '모닝죽', '모닝죽', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
몇년째
['아침식사', '모닝죽', '모닝죽', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
모닝죽
['아침식사', '모닝죽', '모닝죽', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
아침
['아침식사', '모닝죽', '모닝죽', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
기도
['아침식사', '모닝죽', '모닝죽', '인테이크', '몇년째', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛', '종류', '단호박', '꿀고구마']
입맛
['

IndexError: list assignment index out of range

In [10]:
intake.dataset[0]

['lora.study',
 '2018-07-03T23:57:47.000Z',
 'lora.study 아침식사 죽 모닝죽 단호박 인테이크\n.\n몇년째 먹는 모닝죽\n아침엔 바쁘기도 하고 입맛도 없는데 요거하나면 아침뚝딱�\n맛은 여러종류가 있는데.. 다 맛있는데 굳이 고르자면 단호박이랑 새로 나온 꿀고구마가 맛있오�',
 ['아침식사',
  '모닝죽',
  '모닝죽',
  '인테이크',
  '몇년째',
  '모닝죽',
  '아침',
  '기도',
  '입맛',
  '하나',
  '아침',
  '뚝딱',
  '맛',
  '종류',
  '모닝죽',
  '모닝죽'],
 ['아침식사',
  '죽',
  '모닝죽',
  '단호박',
  '인테이크',
  '맛스타그램',
  '먹스타그램',
  '식사대용',
  '다이어트',
  '두통',
  '좋아요반사',
  '맛집',
  '간편식',
  '아점',
  '아픔',
  '모닝죽',
  '맛스타',
  '선팔',
  '맞팔',
  '먹스타',
  '음식',
  '푸드스타그램',
  '좋반'],
 ['lora.study#맛스타그램#lfl#foodporn#먹스타그램#식사대용#다이어트#두통#좋아요반사#instant#맛집#간편식#아점#fff#아픔#선팔은맞팔#맛스타#선팔#fastfood#맞팔#meal#먹스타#음식#푸드스타그램#brunch#좋반',
  'orado912선팔눌렀어요~',
  'pty119멋지네요😺',
  'jobodas좋아요 꾹 클릭하고 갈께요~ ^^'],
 '162',
 'https://www.instagram.com/p/Bkye0hsFHJB/?hl=ko&tagged=%EC%9D%B8%ED%85%8C%EC%9D%B4%ED%81%AC']

In [29]:
intake_test.dataset_transfer()

TypeError: list indices must be integers or slices, not str

In [ ]:
list_dic = [{'main':'아침식단', ['식단기록', '다이어트식단', '다이어트']]
def substitution(self, list_dic, field):
    target_list = []
    target_dic = {}
    for i in enuemrate(list_dic):
        sub_words = i[field]
        target_list.extend(sub_words)
        for j in i[sub_words]:
            target_dic[j] = i['main']

    for i in dataset:
        for j in target_list:
            if j in i[field]:
                i[field][idx] = j.replace(j, target_dic[j])
    return dataset

In [3]:
intake.morph_frequency = analyzer.frequency(intake.morph_merged)

In [4]:
intake.morph_frequency

[('인테이크', 5328),
 ('다이어트', 1949),
 ('밀스', 1336),
 ('아침', 1158),
 ('맛', 1133),
 ('모닝죽', 1093),
 ('다이어터', 903),
 ('일상', 738),
 ('다이어트식단', 719),
 ('저녁', 582),
 ('간식', 577),
 ('먹스타그램', 573),
 ('운동', 534),
 ('점심', 515),
 ('오늘', 511),
 ('너무', 484),
 ('식사대용', 456),
 ('다이어트그램', 437),
 ('식단일기', 378),
 ('식단', 328),
 ('씨씨앙', 322),
 ('먹기', 321),
 ('소통', 321),
 ('단호박', 320),
 ('고구마', 315),
 ('우유', 311),
 ('건강', 308),
 ('맛스타그램', 304),
 ('데일리', 294),
 ('밥', 290),
 ('두유', 266),
 ('식단조절', 266),
 ('물', 262),
 ('이벤트', 250),
 ('미래식사', 245),
 ('칼로리', 244),
 ('하나', 243),
 ('식사', 242),
 ('밀스라이트', 235),
 ('하루', 233),
 ('라이트', 227),
 ('아침식사', 221),
 ('맞팔', 218),
 ('문의', 218),
 ('튜닝', 209),
 ('시간', 208),
 ('견과류', 208),
 ('요즘', 204),
 ('구매', 202),
 ('테이크', 200),
 ('단호박죽', 200),
 ('좋아요', 194),
 ('까지', 193),
 ('제품', 193),
 ('다이어트일기', 189),
 ('느낌', 188),
 ('진짜', 186),
 ('식이조절', 183),
 ('모닝', 181),
 ('간편식', 180),
 ('어제', 179),
 ('아침에', 179),
 ('곤약젤리', 178),
 ('이제', 176),
 ('사과', 175),
 ('비타민', 173),
 ('아침밥', 172),
 

In [5]:
intake.LDA = analyzer.SB_LDA()
intake.LDA.make_lda(intake.morph_joined, ntopic=10)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
소통            운동            식단일기          비타민           너무            
맞팔            곤약젤리          점심            다이어트그램        간식            
먹스타그램         야식            다이어트그램        먹스타그램         저녁            
데일리           4칼로리          저녁            과자            하나            
좋아요           헬스타그램         다이어트일기        구매            식단            
선팔            맛있는곤약젤리       식단            쿠키            오늘            
맛스타그램         식단조절          오늘            건강            두유            
견과류           오늘뭐먹지         식이조절          맛스타그램         운동            
간식            스타터키트         식단조절          다이어트쿠키        다이어트그램        
선팔하면맞팔        식이조절          두유            다이어트간식        칼로리           
건강            꿀맛젤리          식단기록          다이어트칩         그냥            
닥터넛츠          입이심심          모닝수프          맛있는다이어트쿠키     느낌            
인친    

In [1]:
topic0 = intake.LDA.related_doc(intake.main_text_list, 0)

NameError: name 'intake' is not defined

In [ ]:
topic1 = intake.LDA.related_doc(intake.main_text_list, 1)

topic2 = intake.LDA.related_doc(intake.main_text_list, 2)

topic3 = intake.LDA.related_doc(intake.main_text_list, 3)

topic7 = intake.LDA.related_doc(intake.main_text_list, 7)

topic9 = intake.LDA.related_doc(intake.main_text_list, 9)

topic3 = intake.LDA.related_doc(intake.main_text_list, 3)

In [5]:
for i in intake.morph_list[:2]:
         print(i, '\n')

['인테이크', '귀리우유', '모닝귀리', '인테이크밀스', '다이어트식품', '다이어트', '리뷰', '리뷰스타그램', '살이', '너무', '다이어트', '시작', '귀리우유', '다이어트', '광고', '너무', '몸', '직접', '실험', '아침식사는', '귀리우유', '점심은', '평상시', '대로', '밀스라이트', '퇴근', '스포', '애니', '시간', '정도', '운동', '밀스라이트', '이렇게', '일주일', '진행', '몸', '변화', '관찰', '리뷰', '다시', '작성'] 

['인테이크', '마차라떼', '녹차라떼'] 



In [8]:
intake.SB_word2vec = analyzer.SB_Word2Vec(intake.hashtags_appended)

intake.flavor = intake.SB_word2vec.get_sim_sen('맛', intake.main_hash_dic, intake.main_text_list, 4)

'''
intake.diet = intake.SB_word2vec.get_sim_sen('다이어트', intake.main_hash_dic, intake.main_text_list, 4)


intake.health = intake.SB_word2vec.get_sim_sen('건강', intake.main_hash_dic, intake.main_text_list, 4)


intake.exercise = intake.SB_word2vec.get_sim_sen('운동', intake.main_hash_dic, intake.main_text_list, 4)


intake.morningjuk = intake.SB_word2vec.get_sim_sen('모닝죽', intake.main_hash_dic, intake.main_text_list, 4)


intake.meals = intake.SB_word2vec.get_sim_sen('밀스', intake.main_hash_dic, intake.main_text_list, 4)


intake.gonyak = intake.SB_word2vec.get_sim_sen('곤약젤리', intake.main_hash_dic, intake.main_text_list, 4)
'''

1.  sujin.nv아침밥 대용으로 모닝죽 꿀꺽꿀꺽��
 인테이크  모닝죽  꿀고구마
154

2.  love_07.30몸이 무거울땐 가볍게 먹자고 주문한건데_
이건뭐 간식수준_ �
 식사대용 아침밥대신 가볍게 먹을라한건데
맛있잖아 _ 양이 너무 작아 결국 컵라면 뜯었음
딸램이도 두봉이나 흡입 _ 더 시키자�
.
.
.
.
 데일리그램 데일리 먹스타그램 맛스타그램 호박죽 단호박죽 모닝죽 인테이크 로켓배송 쿠팡 맛있다 아침 저녁 다이어트 유지어터 간식 디저트 주부놀이 쇼핑 애스타그램 딸스타그램 럽스타그램 줌마그램 일상 소통 선팔 맞팔
2283

3.  onsemiro_love인테이크 모닝죽 배송도착! 오예쁜데?
-
 남친  맛나다 라고 한다  모닝죽  죽  짜먹는죽  인테이크  모닝  단팥죽  단호박죽  고구마죽  3종세트  파우치팩  죽파우치  간편  아침  아침식사  아침대용  아침밥  다이어트  간편죽  알록달록 예쁘다 위메프에서  착한가격 구입  먹어보고 더살지 결정하라고 해야지 ~
 먹스타그램  먹방  죽스타그램  선팔  맞팔  선팔하면맞팔  onsemiro_love
2037

4.  love.sojun 등센서 소울이때문에  아기띠와한몸 인 요즘
간편하게 먹기좋은  아침
그릇에 덜어먹을필요없이 들고 그냥 춥춥 마셔주면 끝 �
.
.
 인테이크  죽  인테이크모닝죽  모닝죽단호박  단호박죽  아침식사  아침식사대용  맛있다  세상에서가장간편한아침  맘스타그램  일상  소통  맞팔  인친  육아스타그램  육아소통  아들딸맘  좋아요  등센서아기  건강식품
2652



"\nintake.diet = intake.SB_word2vec.get_sim_sen('다이어트', intake.main_hash_dic, intake.main_text_list, 4)\n\n\nintake.health = intake.SB_word2vec.get_sim_sen('건강', intake.main_hash_dic, intake.main_text_list, 4)\n\n\nintake.exercise = intake.SB_word2vec.get_sim_sen('운동', intake.main_hash_dic, intake.main_text_list, 4)\n\n\nintake.morningjuk = intake.SB_word2vec.get_sim_sen('모닝죽', intake.main_hash_dic, intake.main_text_list, 4)\n\n\nintake.meals = intake.SB_word2vec.get_sim_sen('밀스', intake.main_hash_dic, intake.main_text_list, 4)\n\n\nintake.gonyak = intake.SB_word2vec.get_sim_sen('곤약젤리', intake.main_hash_dic, intake.main_text_list, 4)\n"

In [9]:
# 클래스 인스턴스가 지금 intake만 형성되어 있어서 에러
tfidf = analyzer.SB_Tfidf([intake.morph_merged, labnosh.morph_merged, meals.morph_merged, morningjuk.morph_merged, easy_food.morph_merged, conven_food.morph_merged])

tfidf_of_all = tfidf.tfidf_hangul

for i in tfidf_of_all:
    pprint(i[:10])
    print()

NameError: name 'labnosh' is not defined

In [1]:
labnosh = Social_analysis('labnosh.txt', 'labnosh.official')

meals = Social_analysis('meals.txt', 'intakefoods')

morningjuk = Social_analysis('morningjuk.txt', 'intakefoods')

easy_food = Social_analysis('easy_food.txt')

conven_food = Social_analysis('conven_food.txt')

keywords = [intake, labnosh, meals, morningjuk, easy_food, conven_food]

### 빈도분석

for i in keywords:
    print(len(i.main_text_list), len(i.morph_merged))

labnosh.morph_frequency = analyzer.frequency(labnosh.morph_merged)
pprint(labnosh.morph_frequency)

meals.morph_frequency = analyzer.frequency(meals.morph_merged)
pprint(meals.morph_frequency)

morningjuk.morph_frequency = analyzer.frequency(morningjuk.morph_merged)
pprint(morningjuk.morph_frequency)

easy_food.morph_frequency = analyzer.frequency(easy_food.morph_merged)
pprint(easy_food.morph_frequency)

conven_food.morph_frequency = analyzer.frequency(conven_food.morph_merged)
pprint(conven_food.morph_frequency)

### 토픽모델링

intake.LDA = analyzer.SB_LDA()
intake.LDA.make_lda(intake.morph_joined, ntopic=10)

topic0 = intake.LDA.related_doc(intake.main_text_list, 0)

topic1 = intake.LDA.related_doc(intake.main_text_list, 1)

topic2 = intake.LDA.related_doc(intake.main_text_list, 2)

topic3 = intake.LDA.related_doc(intake.main_text_list, 3)

topic7 = intake.LDA.related_doc(intake.main_text_list, 7)

topic9 = intake.LDA.related_doc(intake.main_text_list, 9)

topic3 = intake.LDA.related_doc(intake.main_text_list, 3)



labnosh.LDA = analyzer.SB_LDA()
labnosh.LDA.make_lda(labnosh.morph_joined, ntopic=10)

meals.LDA = analyzer.SB_LDA()
meals.LDA.make_lda(meals.morph_joined, ntopic=10)

morningjuk.LDA = analyzer.SB_LDA()
morningjuk.LDA.make_lda(morningjuk.morph_joined, ntopic=10)

easy_food.LDA = analyzer.SB_LDA()
easy_food.LDA.make_lda(easy_food.morph_joined, ntopic=10)

conven_food.LDA = analyzer.SB_LDA()
conven_food.LDA.make_lda(conven_food.morph_joined, ntopic=10)

### Word2Vec

intake.SB_word2vec = analyzer.SB_Word2Vec(intake.hashtags_appended)

intake.flavor = intake.SB_word2vec.get_sim_sen('맛', intake.main_hash_dic, intake.main_text_list, 4)


intake.diet = intake.SB_word2vec.get_sim_sen('다이어트', intake.main_hash_dic, intake.main_text_list, 4)


intake.health = intake.SB_word2vec.get_sim_sen('건강', intake.main_hash_dic, intake.main_text_list, 4)


intake.exercise = intake.SB_word2vec.get_sim_sen('운동', intake.main_hash_dic, intake.main_text_list, 4)


intake.morningjuk = intake.SB_word2vec.get_sim_sen('모닝죽', intake.main_hash_dic, intake.main_text_list, 4)


intake.meals = intake.SB_word2vec.get_sim_sen('밀스', intake.main_hash_dic, intake.main_text_list, 4)


intake.gonyak = intake.SB_word2vec.get_sim_sen('곤약젤리', intake.main_hash_dic, intake.main_text_list, 4)




### TFIDF

tfidf = analyzer.SB_Tfidf([intake.morph_merged, labnosh.morph_merged, meals.morph_merged, morningjuk.morph_merged, easy_food.morph_merged, conven_food.morph_merged])

tfidf_of_all = tfidf.tfidf_hangul

for i in tfidf_of_all:
    pprint(i[:10])
    print()

meals_juk_SB_Tfidf = analyzer.SB_Tfidf([meals.morph_merged, morningjuk.morph_merged])

tfidf_meals_juk = meals_juk_SB_Tfidf.tfidf_hangul

for i in tfidf_meals_juk:
    pprint(i[:10])
    print()

intake_meals_SB_Tfidf = analyzer.SB_Tfidf([intake.morph_merged, meals.morph_merged])

tfidf_intake_meals = intake_meals_SB_Tfidf.tfidf_hangul

for i in tfidf_intake_meals:
    pprint(i[:10])
    print()

intake_morningjuk_SB_Tfidf = analyzer.SB_Tfidf([intake.morph_merged, morningjuk.morph_merged])

tfidf_intake_morningjuk = intake_morningjuk_SB_Tfidf.tfidf_hangul

for i in tfidf_intake_morningjuk:
    pprint(i[:10])
    print()

meals_conven_food_SB_Tfidf = analyzer.SB_Tfidf([meals.morph_merged, conven_food.morph_merged])

tfidf_meals_conven_food = meals_conven_food_SB_Tfidf.tfidf_hangul

for i in tfidf_meals_conven_food:
    pprint(i[:10])
    print()

morningjuk_conven_food_SB_Tfidf = analyzer.SB_Tfidf([morningjuk.morph_merged, conven_food.morph_merged])

tfidf_morningjuk_conven_food = morningjuk_conven_food_SB_Tfidf.tfidf_hangul

for i in tfidf_meals_conven_food:
    pprint(i[:10])
    print()

C:\Users\abys\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
import csv
with open('morph_list.csv', 'w', encoding='utf-8') as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in intake.morph_list:
        spamwriter.writerow(i)

In [16]:
def morph_pos(text_list, add_list, del_list, exception_list = ['맛', '밥', '물', '몸']):
    morph_list = []
    noun_list = []
    adj_list = []
    verb_list = []

    for j in text_list:
        parsed = self.twitter.pos(j)
        temp = []
        n_temp = []
        adj_temp = []
        verb_temp = []

        for i in parsed:
            if self.isHangul(i):
                if not i[0] in del_list:
                    if len(i[0]) > 1:
                        temp.append(i)
                        if i[1] == 'Noun':
                            n_temp.append(i[0])
                        elif i[1] == 'Verb':
                            n_adj.append(i[0])
                        elif i[1] == 'Adjective':
                            n_verb.append(i[0])

                    elif i in exception_list:
                        temp.append(i)
            else: print(i, '한글이 아닙니다.')

        morph_list.append(temp)
        noun_list.append(n_temp)
        adj_list.append(adj_temp)
        verb_list.append(verb_temp)


    nav_list = noun_list + adj_list + verb_list

    return morph_list, nav_list, noun_list, adj_list, verb_list

In [ ]:

    '''def get_from_dic(self, filename):
        self.data = handler.load_by_pickle(filename)
        self.dataset, self.dataset_dic, self.main_text_list, self.morph_list, self.morph_merged, self.morph_joined, self.hashtags_merged, self.hashtags_appended, self.main_hash_dic = handler.create_datasets_with_dic(self.data)

        # dataset, main_text_list, morph_list, morph_merged, morph_joined, hashtags_merged, hashtags_appended, main_hash_dic
    def get_data_from_list(self, filename, add_list, del_list):
        self.data = handler.load_by_pickle(filename)
        self.dataset, self.main_text_list, self.morph_list, self.morph_merged, self.morph_joined, self.hashtags_merged, self.hashtags_appended, self.main_hash_dic = handler.create_datasets_with_list(self.data[1:], add_list, del_list)'''
